# ICD - coding the admission diagnosis dataset

In [23]:
import pandas as pd
import numpy as np

In [24]:
import pandas as pd

patient_icd_codes = pd.read_csv("icd.csv")
patient_icd_codes.head()

,patient_id,icd9_code
0,OaZ5rutUDcw,"410.41, I21.19"
1,NfOdacqZph8,518.82
2,f@cnrPAqq82,"344.00, G82.50"
3,jCMi7vSIBnk,358.00
4,8Vxltnzo29Q,518.82


In [27]:
import pandas as pd
import numpy as np

# Note, I have named the file "icd_10_allcodes_allpatients.csv"
icd_codes = pd.read_csv("icd_10_allcodes_allpatients.csv")

# Preview the file
icd_codes.head(20)

,patient_id,icd9_code,seq_num,diagnosispriority
0,hVmkjHzfu4!,"414.00, I25.10",NaN,Other
1,hVmkjHzfu4!,"491.20, J44.9",NaN,Other
2,hVmkjHzfu4!,"491.20, J44.9",NaN,Other
3,hVmkjHzfu4!,"428.0, I50.9",NaN,Other
4,hVmkjHzfu4!,"427.31, I48.0",NaN,Primary
5,hVmkjHzfu4!,"585.9, N18.9",NaN,Other
6,hVmkjHzfu4!,"414.00, I25.10",NaN,Other
7,hVmkjHzfu4!,"428.0, I50.9",NaN,Other
8,hVmkjHzfu4!,"585.9, N18.9",NaN,Other
9,hVmkjHzfu4!,"427.31, I48.0",NaN,Primary


* Note the column "icd9_code" - The first item in each column is the ICD9 code, followed by the ICD10 code
* I want to extract **just the ICD9 code**
    * *Note this is actually, ICD-9-CM code* - specific to Centres for Medicare and Medicaid - it is not the international form of the ICD9 code

In [28]:
## Split the icd9_code column into separate columns
icd_codes_split = icd_codes['icd9_code'].str.split(pat=',', expand=True)

# Examine how the dataset looks after splitting
display(icd_codes_split.head())

,0,1,2,3,4,5,6
0,414.00,I25.10,None,None,None,None,None
1,491.20,J44.9,None,None,None,None,None
2,491.20,J44.9,None,None,None,None,None
3,428.0,I50.9,None,None,None,None,None
4,427.31,I48.0,None,None,None,None,None


In [29]:
# We will take only the first column of the above dataframe as the addmission diagnosis (these are mostly as ICD9 codes)
# We will append this column onto the imported dataset above as "icd9"

icd_codes["icd9"] = icd_codes_split.iloc[:,0]
icd_codes.head()

,patient_id,icd9_code,seq_num,diagnosispriority,icd9
0,hVmkjHzfu4!,"414.00, I25.10",NaN,Other,414.00
1,hVmkjHzfu4!,"491.20, J44.9",NaN,Other,491.20
2,hVmkjHzfu4!,"491.20, J44.9",NaN,Other,491.20
3,hVmkjHzfu4!,"428.0, I50.9",NaN,Other,428.0
4,hVmkjHzfu4!,"427.31, I48.0",NaN,Primary,427.31


In [30]:
# clean the dataset 
# Replace sequence number with prioritisation - this step is not really necessary
# I will not be using priority as a feature in the model
# I am only using the codes to calculate the Charlson Index

icd_codes.loc[icd_codes["seq_num"] == 1,"diagnosispriority"] = "Primary"
icd_codes.loc[icd_codes["seq_num"] == 2,"diagnosispriority"] = "Major"
icd_codes.loc[icd_codes["seq_num"] > 2,"diagnosispriority"] = "Other"

In [31]:
# Preview the datasets

display(icd_codes.head())
display(icd_codes.tail())

,patient_id,icd9_code,seq_num,diagnosispriority,icd9
0,hVmkjHzfu4!,"414.00, I25.10",NaN,Other,414.00
1,hVmkjHzfu4!,"491.20, J44.9",NaN,Other,491.20
2,hVmkjHzfu4!,"491.20, J44.9",NaN,Other,491.20
3,hVmkjHzfu4!,"428.0, I50.9",NaN,Other,428.0
4,hVmkjHzfu4!,"427.31, I48.0",NaN,Primary,427.31


,patient_id,icd9_code,seq_num,diagnosispriority,icd9
856308,KgOZfKRVOfC,V4582,9.0,Other,V4582
856309,KgOZfKRVOfC,78820,10.0,Other,78820
856310,KgOZfKRVOfC,28529,11.0,Other,28529
856311,KgOZfKRVOfC,V4986,12.0,Other,V4986
856312,KgOZfKRVOfC,4293,13.0,Other,4293


In [32]:
# Now identify all rows that are not correctly coded ICD9 codes
# We will use a Regex pattern to match properly identified codes
# If a row matches the pattern, it will be classified as TRUE on a new column called "Correct_code"

import re

pattern1 = re.compile("^(V\d{2}(\.\d{1,2})?|(\s{0,1})?V\d{2}(\d{1,2})?(\s{0,1})?|\d{3}(\.\d{1,2})?|\d{3}(\d{1,2})?|(\s{0,1})?E\d{3}(\.\d)?|E\d{3}(\d)?(\s{0,1})?)$")
icd_codes['correct_code'] = icd_codes.icd9.apply(lambda x: bool(pattern1.match(x)))

pattern2 = re.compile("^[0-9]{2}$")
icd_codes.loc[icd_codes['correct_code']==False,'correct_code'] = icd_codes.icd9.apply(lambda x: bool(pattern2.match(x)))

In [33]:
# Display the dataset

icd_codes.head()

,patient_id,icd9_code,seq_num,diagnosispriority,icd9,correct_code
0,hVmkjHzfu4!,"414.00, I25.10",NaN,Other,414.00,True
1,hVmkjHzfu4!,"491.20, J44.9",NaN,Other,491.20,True
2,hVmkjHzfu4!,"491.20, J44.9",NaN,Other,491.20,True
3,hVmkjHzfu4!,"428.0, I50.9",NaN,Other,428.0,True
4,hVmkjHzfu4!,"427.31, I48.0",NaN,Primary,427.31,True


In [34]:
# Now count how many rows are NOT in the right ICD9 format
# We can see there are over 7500 rows

(icd_codes['correct_code'] == False).sum()

7587

In [35]:
# Drop unneeded columns

icd_codes.drop(["icd9_code","seq_num"], axis=1, inplace=True)

In [36]:
icd_codes.head()

,patient_id,diagnosispriority,icd9,correct_code
0,hVmkjHzfu4!,Other,414.00,True
1,hVmkjHzfu4!,Other,491.20,True
2,hVmkjHzfu4!,Other,491.20,True
3,hVmkjHzfu4!,Other,428.0,True
4,hVmkjHzfu4!,Primary,427.31,True


In [37]:
# Save the current output file as a CSV (if wanting to examine wrongly coded rows in Excel)

icd_codes.to_csv("icd_allpatients_interm.csv")

In [38]:
# Let's view those rows which have incorrectly formatted codes
# We can see that these are actually ICD10 codes

icd_codes.loc[icd_codes['correct_code']==False,"icd9"].apply(lambda x: len(x))
icd_codes.loc[icd_codes['correct_code']==False, "icd9"]

127         I42.0
276         I42.0
279         I42.0
280         I42.0
282         I42.0
291         I42.0
404         K65.1
422          31.1
424          31.1
862         I42.0
863         I42.0
864         I42.0
865         E87.1
942           S02
943           S02
957         S10.8
973         S22.4
975         S22.4
976         S22.4
979         S22.4
1114        S02.1
1261         31.1
1367          S02
1371          S02
1640      F19.939
1644        S22.4
1645        S22.4
1646        S22.4
1653        S22.4
1658        S22.4
           ...   
808806      V08  
808814      V08  
809617      V08  
811512      V08  
812038      V08  
812085      V08  
812188      V08  
817975      V08  
820829      V08  
828003      V08  
828171      V08  
831318      V08  
832105      V08  
832540      V08  
833562      V08  
834312      V08  
839421      V08  
843887      V08  
844149      V08  
844492      V08  
845424      V08  
845637      V08  
846583      V08  
847658      V08  
850233    

In [39]:
# I will import an ICD10 to ICD9 dictionary that shows corresponding ICD9 and ICD10 version codes
icd10to9 = pd.read_csv("icd9to10dictionary.txt", sep = "|", header=None)

# Give names to the columns (the column "icd9" is actually an ICD10 code - I named it icd9 because this column
# will be used for merging to the admission codes dataframe)
icd10to9.columns = ['icd9_code','icd9','description']
display(icd10to9.head())


,icd9_code,icd9,description
0,001.0,A00.0,Cholera due to Vibrio cholerae 01 biovar chole...
1,001.1,A00.1,Cholera due to Vibrio cholerae 01 biovar eltor
2,001.9,A00.9,Cholera unspecified
3,002.0,A01.00,Typhoid fever unspecified
4,002.1,A01.1,Paratyphoid fever A


In [40]:
# Now MERGE this dataset to the admission codes dataframe we have been working with
# We will merge on the "icd9" columns of both datasets
# Only rows having an ICD10 code (for the icd_codes data) will match to the icd10to9 dataset above

merged = pd.merge(left = icd_codes, right = icd10to9, on = 'icd9', how='left')
merged.head()

,patient_id,diagnosispriority,icd9,correct_code,icd9_code,description
0,hVmkjHzfu4!,Other,414.00,True,NaN,NaN
1,hVmkjHzfu4!,Other,491.20,True,NaN,NaN
2,hVmkjHzfu4!,Other,491.20,True,NaN,NaN
3,hVmkjHzfu4!,Other,428.0,True,NaN,NaN
4,hVmkjHzfu4!,Primary,427.31,True,NaN,NaN


In [41]:
# We will create a filter that identifies only those with non-missing values on the icd9_code column
# Anyone NOT missing a value for the ICD9 code column are those currently with an ICD10 code
# We use the np.where function to replace the icd10 code with the corresponding icd9 code

filt = merged["icd9_code"].notna()
merged['icd9'] = np.where(filt,merged['icd9_code'],merged['icd9'])

In [42]:
# Calculate regex filter agains - and then count how many rows we have invalid values

len(icd_codes.loc[0,"icd9"])

# Regex pattern to match properly identified codes

import re

pattern1 = re.compile("^(V\d{2}(\.\d{1,2})?|(\s{0,1})?V\d{2}(\d{1,2})?(\s{0,1})?|\d{3}(\.\d{1,2})?|\d{3}(\d{1,2})?|(\s{0,1})?E\d{3}(\.\d)?|E\d{3}(\d)?(\s{0,1})?)$")
merged['correct_code'] = merged.icd9.apply(lambda x: bool(pattern1.match(x)))
pattern2 = re.compile("^[0-9]{2}$")
merged.loc[merged['correct_code']==False,'correct_code'] = merged.icd9.apply(lambda x: bool(pattern2.match(x)))
pattern3 = re.compile("^[0-9]{2}\.[0-9]$")
merged.loc[merged['correct_code']==False,'correct_code'] = merged.icd9.apply(lambda x: bool(pattern3.match(x)))
(merged['correct_code'] == False).sum()

3809

In [43]:
# Below, list all those incorrectly labelled codes
merged['icd9'][merged['correct_code'] == False].unique()

array(['I42.0', 'S02', 'S10.8', 'S22.4', 'S02.1', 'S32.00', 'T51.0',
       'S22.00', 'S12', 'T40.60', 'K61.1', 'F17.20', 'T43.01', 'I68.0',
       'S24.1', 'G91.9', 'S42.00', 'G40.1', 'S32.1', 'S22.2', 'S42.10',
       'S14.1', 'S73.00', 'I35.0', 'I37.1', 'S36.89', 'S32.30', 'S32.59',
       'S32.40', 'S32.60', 'T81.7', 'S36.50', 'S27.81', 'V08  '],
      dtype=object)

In [44]:
# Manually replace those codes with an inferred ICD9 code (the ICD9 code closest to that value)

wrong = ['I42.0',
 'S02',
 'S22.4',
 'S02.1',
 'S32.00',
 'S22.00',
 'S12',
 'F17.20',
 'T43.01',
 'S24.1',
 'S42.00',
 'G40.1',
 'S32.1',
 'S22.2',
 'S42.10',
 'S14.1',
 'S36.89',
 'S32.30',
 'S32.40',
 'S32.60',
 'T81.7',
 'S36.50',
 'S27.81',
 'V08',
    '31.1']

correct = ['425',
'800',
'807',
'801',
'805.4',
'805.2',
'806',
'305.2',
'969.05',
'806.2',
'810',
'345.5',
'805.6',
'807.2',
'V54.11',
'952',
'863.85',
'808.41',
'808',
'808.42',
'997.2',
'863.4',
'862.22',
'V080',
'031.1']

# Do the replacements
for wrongcode, correctcode in zip(wrong, correct):
    merged['icd9'] = merged['icd9'].replace({wrongcode:correctcode})

    
# Some codes are invalid because they do not have the padded 0s. So we add these to those particular codes
# Add padded 0's to the 3-digit codes
merged['icd9'] = merged['icd9'].apply(lambda x: "0" + x if len(x) == 2 else x)

# Add padded 0's to the 2-digit codes
merged['icd9'] = merged['icd9'].apply(lambda x: "0" + x if len(x) == 2 else x)

In [46]:
# Check again, how many rows are invalid

pattern1 = re.compile("^(V\d{2}(\.\d{1,2})?|(\s{0,1})?V\d{2}(\d{1,2})?(\s{0,1})?|\d{3}(\.\d{1,2})?|\d{3}(\d{1,2})?|(\s{0,1})?E\d{3}(\.\d)?|E\d{3}(\d)?(\s{0,1})?)$")
merged['correct_code'] = merged.icd9.apply(lambda x: bool(pattern1.match(x)))
pattern2 = re.compile("^[0-9]{2}$")
merged.loc[merged['correct_code']==False,'correct_code'] = merged.icd9.apply(lambda x: bool(pattern2.match(x)))
pattern3 = re.compile("^[0-9]{2}\.[0-9]$")
merged.loc[merged['correct_code']==False,'correct_code'] = merged.icd9.apply(lambda x: bool(pattern3.match(x)))
(merged['correct_code'] == False).sum()

# We will drop these 775 rows (which is a lot better than dropping 7500 rows)

775

In [47]:
# strip all codes of white space
merged['icd9'] = merged['icd9'].apply(lambda x: x.strip())

In [51]:
# Drop any duplicate rows
merged_no_dups = merged.drop_duplicates()

# Count how many rows are duplicates
merged_no_dups.duplicated().sum()

0

In [52]:
# Calculate a column that counts the rows within each patient (i.e., it increments by 1, with each additional record)
merged_no_dups["int"]=1
merged_no_dups["record"] = merged_no_dups.groupby("patient_id")["int"].transform(np.cumsum)
merged_no_dups.drop("int", axis=1, inplace=True)

C:\Users\z3160256\.conda\envs\insight\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\z3160256\.conda\envs\insight\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\z3160256\.conda\envs\insight\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [56]:
merged_no_dups

,patient_id,diagnosispriority,icd9,correct_code,description,record
0,hVmkjHzfu4!,Other,414.00,True,NaN,1
1,hVmkjHzfu4!,Other,491.20,True,NaN,2
3,hVmkjHzfu4!,Other,428.0,True,NaN,3
4,hVmkjHzfu4!,Primary,427.31,True,NaN,4
5,hVmkjHzfu4!,Other,585.9,True,NaN,5
10,5iSvqTenmIT,Other,456.0,True,NaN,1
11,5iSvqTenmIT,Other,584.9,True,NaN,2
12,5iSvqTenmIT,Other,595.9,True,NaN,3
13,5iSvqTenmIT,Other,425.8,True,NaN,4
14,5iSvqTenmIT,Other,780.57,True,NaN,5


In [57]:
# drop unneeded columns
merged_no_dups.drop(['diagnosispriority','correct_code','description'], axis=1, inplace=True)

In [58]:
# Count how many patients are in this cleaned admission codes dataset
len(merged_no_dups.patient_id.unique())

43105

In [59]:
# What does the dataset look like?
merged_no_dups.head(20)


,patient_id,icd9,record
0,hVmkjHzfu4!,414.00,1
1,hVmkjHzfu4!,491.20,2
3,hVmkjHzfu4!,428.0,3
4,hVmkjHzfu4!,427.31,4
5,hVmkjHzfu4!,585.9,5
10,5iSvqTenmIT,456.0,1
11,5iSvqTenmIT,584.9,2
12,5iSvqTenmIT,595.9,3
13,5iSvqTenmIT,425.8,4
14,5iSvqTenmIT,780.57,5


In [233]:
# IMPORTANT - SAVE THIS FILE 
# We will open this file in Stata to put ALL codes in the ic9 column in the correct consistent format
merged_no_dups.to_csv("icd_allpatients.csv")

# STATA

* Stata was used to make all the codes in the icd9 column above in a UNIFORM "XXX.XX" format.
The code is pasted below:
* The syntax is contained in the file icd9codes.do (this is a syntax file that can be opened in Stata)
* The cleaned Stata dataset is contained in the file: sepsis_patient_correct_codes-long.dta

In [ ]:
################### THIS IS STATA CODE #####################


import delimited "C:\Users\z3160256\OneDrive - UNSW\R Data Camp\icd_allpatients.csv", delimiter(comma) varnames(1) clear 
drop v1
rename (icd9) (dx)

icd9 check dx
icd9 check dx, generate(prob)
generate anyprob=prob>0

*examine result - sorts results and calculates number of problems
by patient_id, sort: egen numprobs=total(anyprob)

*drop prob and any prob
drop prob anyprob

*drop all invalid rows
drop if numprobs > 0

*clean the diagnosis codes
icd9 clean dx, dots pad

drop record

*reshape dataset
reshape wide dx, i(patient_id) j(record)

*calculate charlson
charlson, icd(9) idvar(patient_id) cmbprfx(dx)

*save file
save "C:\Users\z3160256\OneDrive - UNSW\R Data Camp\sepsis_patients_correct_codes.dta", replace

*export wide version
export delimited using "C:\Users\z3160256\OneDrive - UNSW\R Data Camp\sepsis_cleaned_icd.csv", replace

*export long version
export delimited using "C:\Users\z3160256\OneDrive - UNSW\R Data Camp\sepsis_cleaned_icd_long.csv", replace

# R
* R was used to use the diagnosis codes to calculate whether each patient has each of 17 categories of chronic conditions
* This information was then used to calculate the Charlson comorbidity index of each patient.
* The code is pasted below, and is contained in the file: **R code for calculating the Charlson index.R**

In [ ]:
################### THIS IS R CODE #######################

# Import the dataset from Stata
sepsis_cleaned_icd_long <- read.csv("C:/Users/z3160256/OneDrive - UNSW/R Data Camp/sepsis_cleaned_icd_long.csv")

# Load needed libraries
library(medicalrisk)
library(plyr)
library(comorbidity)

# Use comorbidity package - this calculates scores on each of the individual comorbidities
comorbidity <- comorbidity(x = sepsis_cleaned_icd_long, id = "patient_id", code = "dx_r", score = "charlson")

count(sepsis_cleaned_icd_long_r, c('dx_r'))

#drop unneeded columns
drops <- c("dx","numprobs","id")
clean <- sepsis_cleaned_icd_long[ , !(names(sepsis_cleaned_icd_long) %in% drops)]
names(clean) <- c("id","icd9cm")

# Use the generate_comorbidity_df function to put data into a format to calculate the CCI
# This is part of the medicalrisk package
comorbid <- generate_comorbidity_df(clean)

# calculate the CCI - this uses the medical risk package
cci_df <- generate_charlson_index_df(comorbid, idvar = "id", weights = medicalrisk::charlson_weights)

# export the CCI score dataset
write.csv(cci_df,"C:/Users/z3160256/OneDrive - UNSW/R Data Camp/charlson.csv", row.names = FALSE)

# Export the comorbidities (TRUE FALSE) dataset
write.csv(comorbid,"C:/Users/z3160256/OneDrive - UNSW/R Data Camp/comorbidities.csv", row.names = FALSE)
